In [1]:
import os
os.chdir('../')
%pwd

'C:\\Users\\VICTUS\\Documents\\Course Material\\Krish-Naik-End-to-End-ML-Project\\Chicken-Disease-Classifier-Project'

In [2]:
import torch

model = torch.load('artifacts/training/trained_model.pt')

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_batch_size: int

In [4]:
import os
from pathlib import Path
from cnn_classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnn_classifier.utils.common import read_yaml, create_directories, save_json
from cnn_classifier.entity.config_entity import DataIngestionConfig, PrepareBaseModelConfig, TrainingConfig

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE
        )
        return training_config
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/trained_model.pt",
            training_data="artifacts/data_ingestion/Chicken-fecal-images/",
            all_params=self.params,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    


In [7]:
from urllib.parse import urlparse
from cnn_classifier import logger
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
from cnn_classifier.utils.common import read_yaml, create_directories, save_json

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.use_cuda = torch.cuda.is_available()

    def get_base_model(self, path):
        self.model = torch.load(path)
        if self.use_cuda:
            self.model = self.model.cuda()

    def get_data_loaders(self):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        data = ImageFolder(self.config.training_data, transform=transform)
        test_ratio = 0.2
        total_size = len(data)
        test_size = int(test_ratio * total_size)
        train_size = total_size - test_size
        batch_size = self.config.params_batch_size
        # Split the dataset into training and testing
        train_dataset, test_dataset = random_split(data, [train_size, test_size])
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
        return train_loader, test_loader

    def evaluation(self):
        self.get_base_model(self.config.path_of_model)
        _, valid_loader = self.get_data_loaders()
        criterion = nn.CrossEntropyLoss()
        # Validation
        self.model.eval()
        correct = 0
        total = 0
        valid_loss = 0
        for batch_idx, (inputs, labels) in enumerate(valid_loader):
            if self.use_cuda:
                inputs, labels = inputs.cuda(), labels.cuda()
            with torch.no_grad():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            accuracy = 100 * correct / total
            logger.info(f"Validation Loss= {valid_loss:.3f} Validation Accuracy = {accuracy:.2f}%")
        score = {"loss": float(valid_loss), "accuracy": accuracy}
        save_json(path=Path("scores.json"), data=score)
        logger.info('Saved Scores')        
        

In [8]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
except Exception as e:
   raise e

[2023-09-24 21:22:07,053]: INFO: common: yaml file: config\config.yaml loaded successfully
[2023-09-24 21:22:07,064]: INFO: common: yaml file: params.yaml loaded successfully
[2023-09-24 21:22:07,068]: INFO: common: created directory at: artifacts
[2023-09-24 21:22:12,172]: INFO: 2233906510: Validation Loss= 0.037 Validation Accuracy = 100.00%
[2023-09-24 21:22:16,235]: INFO: 2233906510: Validation Loss= 0.048 Validation Accuracy = 100.00%
[2023-09-24 21:22:20,210]: INFO: 2233906510: Validation Loss= 0.044 Validation Accuracy = 100.00%
[2023-09-24 21:22:24,199]: INFO: 2233906510: Validation Loss= 0.081 Validation Accuracy = 98.44%
[2023-09-24 21:22:27,714]: INFO: 2233906510: Validation Loss= 0.096 Validation Accuracy = 98.72%
[2023-09-24 21:22:27,717]: INFO: common: json file saved at: scores.json
[2023-09-24 21:22:27,721]: INFO: 2233906510: Saved Scores
